# Collaboration and Competition

---

This repository is based on the collaboration and competition problem studied in the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program and demonstrates a variant of the PPO learning algorithm in a multi-agent environment with a continuous action space.

![Actor](resources/agent.gif)

## 1. General description

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. In each timestep the agent receives the state of the last three time steps and each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

The task is episodic, and in order to solve the environment, the agents must get an average score of +0.5 (over 100 consecutive episodes, after taking the maximum over both agents). Specifically, after each episode, we add up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores. This yields a single score for each episode.

The environment is considered solved, when the average (over 100 episodes) of those scores is at least +0.5.

## 2. PPO Algorithm with adaptions

Given that each agent only has only local observations of the environment, i.e. it does not know the state of the other agent, we followed a naive approach to the problem and just applied a traditional reinforcement algorithm to the problem. We treat both agents completely separate, each only using it's local observation and considering the effects of the other agent as part a (non-stationary) environment. This approach turned out to be capable of handling non-stationarity of the problem sufficiently well and give good results.

### 2.1 Used algorithm and adaptions

The algorithm we used follows closely the algorithm described in section 2 of this [project on continuous control](https://github.com/numblr/drlnd-/blob/master/Report.ipynb#2.-Algorithm). This algorithm is a variant of *Proximal Policy Optimization (PPO)* with *Generalized Advantage Estimation (GAE)* and a clipped objective function. As a critic an estimate of the value function is used. In addition, the algorithm uses a finite horizon instead of reward discounting, which is implemented using sliding windows over an episode. To keep this part short we moved the detailed description of the algorithm to the [Appendix](#5.-Appendix:-Varinant-of-the-PPO-Algorithm) and only discuss the modifications made to the algorithm in this section.

The algorithm described above was successfully applied to a continuous control problem ([Reacher](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#reacher) enviroment) were a double-jointed arm is chasing a moving target. The original task was quasi-continuous, capped to episodes of 1000 timesteps. The original implementation of the algorithm was capable of training on multiple parallel agents, each acting on it's own separate environment. In the following we discuss the adaptions that were made to the algorithm in order to apply it to the current problem:

#### 2.1.1. Exploration vs Exploitation

In the original algorithm the policy on the continuous action space was modeled by a DNN model to estimate the mean of a normal distribution with a given, fixed variance. This model is on the one hand capable of handling the continuous action space and on the other hand takes care of adding exploration to the policy due to the non-zero variance. The problem, to which the algorithm was initally applied, worked well with this simple model and tolerated a relatively high variance without the policy becomming unpredictable.

The problem examined in this project, however, is much more sensitive: High variance leads to the inability to exploit the current policy, i.e. the actions sampled from the policy are too inaccurate to give a good result. Contrary, too low variance leads to a lack of exploration, and the agent quickly get's stuck in a situation were he never encounters a useful action from which it could learn.

This was solved by using a mixture of a normal distribution with high variance for exploration and a normal distribution with low variance for exploitation:

$$
    \pi(a|s) = (1 - \epsilon) \mathcal{N}(\hat{\mu}_\theta(s), \sigma_{\rm exploit})
        + \epsilon \mathcal{N}(\hat{\mu}_\theta(s), \sigma_{\rm explore})
$$


#### 2.1.2 Converting the episodic task to a quasi-continuous task

The original task was quasi-continuous, capped to an episode of 1000 timesteps. The used algorithm implements a finite horizon for the agent by using a sliding window on the episode. Besides providing a finite horizon, this technique also allows to very effectively reduce noise during training by filtering out windows were no positive reward was achieved. To apply this strategy in the current setting again, during training episodes were concatenated to sequences of 1000 timesteps, and terminal states were penalized with a reward of -1 (during training only) to reflect that we want to maximize reward *within a single* episode. 

#### 2.1.3 Model initialization

Initialization of the model parameters for actor and critic was modified to initialize all parameters and bias close to zero. This provides a good starting point for learning in this environment.

With this tweaks the original algorithm could be applied with minimal modification.



### 2.2 Further remarks on the algorithm and chosen hyper parameters

* During training multiple episodes are concatenated to a sequence of 1152 steps.
* During training multiple the reward of terminal states in this sequence is modified to -1.
* The horizon of the agent is limited to 72 steps during the first 128 epochs, to 96 steps until epoch 256 and 128 steps after
* Sliding windows corresponding to the horizon with a offset of 2 steps are extracted, from which windows where no positive reward is obtained in any timestep are filtered out.
* No reward discounting is used as we already limit the horizon of the agent. 
* The surrogate objective clipped with a window size of 0.1.
* The parameter for the GAE is set to 0.1 to provide large variance reduction.
* The DNN models used are simple and have both an input size of 24, hidden layers of size 128 and 64, and and output layer of 2 with tanh activation for the actor net and an output layer of size 1 for the critic. (see also [section](#5.3-Approximators-and-training) of the appendix).
* Parameters and bias in the used DNN models are initilized around zero. 
* After generating 8 sequences of 1152 steps, learning is executed for 2 epochs on the sampled data in batches of size 32 and a learning rate of $10^{-3}$.   
* Gradient clipping is applied to maximize the norm of the gradient to 1.
* The variance for the normal distributions is set to 0.5 for exploration and 0.1 for exploitation, mix with an $\epsilon$-value of 0.25.

### 2.6 Results and performance
The following plot shows the scores for each generated episode during the training process. The plot also includes the average over the last 100 episodes. The average crosses the +0.5 mark around the 25000 episodes and reaches a top of about +0.8. Note, however, that during training we add a significant amount of noise to the policy, thus the scores during training are way below the score the trained agent achieves with the policy without adding noise. For this reason it is also hard to tell at what point the trained agent would solve the environment. 

![Scores](scores.png)

**As shown in the plot, the agent reaches an average score of +0.3 on 100 episodes after about than 25000 episodes during training. The trained agent achieves an average score of around 2.6 on 100 episodes when [executing the policy without noise](#4.6-Evaluation).**

## 2.7 Possible Improvements

The trained agent delivers quite good results and mostly reaches the hard coded limit of 1000 timesteps for the game. Nevertheless, there are quite a couple of points for improvement:

### 2.7.1 Sample efficiency

The first evident point that can be improved is sample efficiency. The main issue with the applied algorithm is that it generates full episodes before learning with a rather low level of reuse. It should be expected that temporal difference methods like DDPG combined with a replay buffer work much more sample efficient, as the can on the one hand improve the policy already during episode execution and on the other hand they can provide reuse through the replay buffer more efficiently. In combination with priority sampling this should also be a good replacement for the windowing strategy applied in this algorithm.

### 2.7.2 Noise

The noise used by the applied algorithm seems to be one of the most crucial points in this problem, as the balance between exploration and exploitation is quites sensitive in this problem. An improvement over the naive model applied in the presented solution could be the application of an [Ornstein-Uhlenbeck](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process) noise process, which is a modified random walk that offers a constant long-term distribution. Such a noise process has the advantage that it provides low variance in the short term, which supports exploitation, as actions remain more predictable, while providing high varince in the long term, which supports exploration.   

### 2.7.3 Non-stationarity

Though non-stationarity of the environment turned out to be not a crucial issue in this problem, is in general a problem in multi-agent environments, so it should be expected that the presented solution does not work as nicely in other environments. A straight forward improvement in this direction would be to extend the critic to depend on the state of both agents in the game to make the training process more stable. 

## 4. Implementation

In this section we provide a detailed walk through the implementation. To reproduce the above results, however, run the *coball.py* script from the command line as described in the README without command line parameters.

First we run some neccessary imports.

In [1]:
%matplotlib inline

import os
import pkg_resources
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions.normal as dist
from torch import optim
from torch import autograd

import numpy as np

from unityagents import UnityEnvironment
from unityagents.exception import UnityEnvironmentException

from coball.util import plot

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PLATFORM_PATHS = ['Tennis.app',
        'Tennis_Windows_x86/Tennis.exe',
        'Tennis_Windows_x86_64/Tennis.exe',
        'Tennis_Linux/Tennis.x86',
        'Tennis_Linux_NoVis/Tennis.x86',
        'Tennis_Linux/Tennis.x86_64',
        'Tennis_Linux_NoVis/Tennis.x86_64']

### 4.1. Start the Environment

The environment the agent operates on is based on a pre-built [Unity ML-Agent](https://github.com/Unity-Technologies/ml-agents) and is provided by the *CoBallEnv* class in the *.environment* package.

This class encapsulates the pre-built Unity environment, which must be downloaded to one of the following locations **_before running the code cells below_**:

- **Mac**: `"coball/resources/Tennis.app"`
- **Windows** (x86): `"coball/resources/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"coball/resources/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"coball/resources/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"coball/resources/Reacher_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"coball/resources/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"coball/resources/Tennis_Linux_NoVis/Tennis.x86_64"`

It is important that the */resources* folder **_only contains the agent for your operating system_**!

If the code cell below returns an error, please revisit the installation instructions in the README and double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
# %load -s CoBallEnv coball/environment
class CoBallEnv:
    """Continuous control environment.

    The environment accepts actions and provides states and rewards in response.
    """

    def __init__(self):
        for path in PLATFORM_PATHS:
            try:
                unity_resource = pkg_resources.resource_filename('coball', 'resources/' + path)
                self._env = UnityEnvironment(file_name=unity_resource)
                print("Environment loaded from " + path)
                break
            except UnityEnvironmentException as e:
                print("Attempted to load " + path + ":")
                print(e)
                print("")
                pass

        if not hasattr(self, '_env'):
            raise Exception("No unity environment found, setup the environment as described in the README.")

        # get the default brain
        self._brain_name = self._env.brain_names[0]
        self._brain = self._env.brains[self._brain_name]

        self._info = None
        self._scores = None
        self._score_history = ()

    def generate_episode(self, agent, max_steps=None, episodic=True, train_mode=False):
        """Create a generator for and episode driven by an actor.
        Args:
            actor: An actor that provides the next action for a given state.
            max_steps: Maximum number of steps (int) to take in the episode. If
                None, the episode is generated until a terminal state is reached.

        Returns:
            A generator providing a tuple of the current state, the action taken,
            the obtained reward, the next state and a flag whether the next
            state is terminal or not.
        """
        states = self.reset(train_mode=train_mode)
        is_terminal = False
        count = 0

        while (max_steps is None or count < max_steps) and (not episodic or not is_terminal):
            actions = agent.act(states)
            rewards, next_states, is_terminals = self.step(actions, episodic)

            step_data = (states, actions, rewards, next_states, is_terminals)

            states = next_states
            is_terminal = np.any(is_terminals)
            count += 1

            if np.any(np.isnan(states)):
                raise ValueError("Nan from env")

            if is_terminal and not episodic and not count == max_steps:
                self._score_history += (np.mean(self._scores), )
                self._scores = np.zeros(self.get_agent_size())

            yield step_data

        self._score_history += (self.get_score(), )
        self._scores = None

    def reset(self, train_mode=False):
        """Reset and initiate a new episode in the environment.

        Args:
            train_mode: Indicate if the environment should be initiated in
                training mode or not.

        Returns:
            The initial state of the episode (np.array).
        """
        # if self._info is not None and not np.any(self._info.local_done):
        #     raise Exception("Env is active, call terminate first")

        if self._scores is not None:
            self._score_history += (np.mean(self._scores), )

        self._info = self._env.reset(train_mode=train_mode)[self._brain_name]
        self._scores = np.zeros(self.get_agent_size())

        return self._info.vector_observations

    def step(self, actions, episodic=True):
        """Execute an action on all instances.

        Args:
            action: An tensor of ints representing the actions each instance.

        Returns:
            A tuple containing the rewards (floats), the next states (np.array) and
            a booleans indicating if the next state is terminal or not.
        """
        if self._info is None:
            raise Exception("Env is not active, call reset first")

        if torch.is_tensor(actions):
            actions = actions.numpy()

        self._info = self._env.step(actions)[self._brain_name]
        next_states = self._info.vector_observations
        rewards = self._info.rewards
        is_terminals = self._info.local_done
        self._scores += np.array(rewards)

        if not episodic:
            rewards = [ -1.0 if x < 0 else x for x in rewards ]

        return rewards, next_states, is_terminals

    def terminate(self):
        self._info = None
        if self._scores is not None:
            self._score_history += (self.get_score(), )
        self._scores = None

    def close(self):
        self.terminate()
        self._env.close()

    def get_score(self):
        """Return the cumulative average reward of the current episode."""
        return np.max(self._scores) if self._scores is not None else self._score_history[-1]

    def get_score_history(self):
        """Return the cumulative average reward of all episodes."""
        return self._score_history

    def clear_score_history(self):
        """Clear the cumulative average reward of all episodes."""
        self._scores = None
        self._score_history = ()

    def get_agent_size(self):
        if self._info is None:
            raise ValueError("No agents are initialized")

        return len(self._info.agents)

    def get_action_size(self):
        return self._brain.vector_action_space_size

    def get_state_size(self):
        return 3 * self._brain.vector_observation_space_size


To generate episodes on the environment the *CoBallAgent* class provides an agent that selects actions in a particular state of the environement based on a policy function:


In [3]:
# %load -s CoBallAgent coball/environment
class CoBallAgent:
    """Agent based on a policy approximator."""

    def __init__(self, policy):
        """Initialize the agent.

        Args:
            pi: policy-function that is callable with n states and returns a
                (n, a)-dim array-like containing the value of each action.
        """
        self._policy = policy

    def act(self, states):
        """Select actions for the given states.

        Args:
            state: An array-like of states to choose the actions for.
        Returns:
            An array-like of floats representing the actions.
        """
        if not torch.is_tensor(states):
            try:
                states = torch.from_numpy(states, dtype=torch.float)
            except:
                states = torch.from_numpy(np.array(states, dtype=np.float))
        else:
            states = states.float()

        with torch.no_grad():
            return self._policy.sample(states)

    def get_policy(self):
        return self._policy


Now we can setup and test the environment and run an episode with a random or dummy policy. To run the test, uncomment the *test_env()* invocation and choose the policy you like.

**_After you ran the test you need to comment it out again and restart the kernel, as recreating the environment does not work!_**

In [4]:
env = CoBallEnv()

def test_env():
    # all actions are between -1 and 1
    class DummyPolicy:
        def sample(self, state):
            return torch.rand(env.get_agent_size(), env.get_action_size()) * 2.0 - 1.0

    agent = CoBallAgent(DummyPolicy())
    episode = enumerate(env.generate_episode(agent, max_steps = 1000))
    for count, step_data in episode:
        # Consume the generated steps
        pass

    env.close()

    
##### UNCOMMENT THIS TO TEST, COMMENT OUT AND RESTART THE KERNEL AFTERWARDS ####

# test_env()


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environment loaded from Tennis.app


### 4.2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
states = env.reset()

# number of agents
num_agents = env.get_agent_size()
print('Number of agents:', num_agents)

# size of each action
action_size = env.get_action_size()
print('Size of each action:', action_size)

# examine the state space 
state_size = env.get_state_size()
print('Size of each states:', state_size)

print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], states.shape[1]))
print('The state for the first agent looks like:', states[0])

env.clear_score_history()

Number of agents: 2
Size of each action: 2
Size of each states: 24
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 4.3 Models and policy

The models to appoximate the action means of the policy distributions (*Actor*) and the state-value function (*Critic*) are implemented as PyTorch *nn.Module*. As they share the same hidden layers, they extend a common base class, and only add their individual output layers.

In [6]:
# %load -s PPOModel coball/model
class PPOModel(nn.Module):
    """Base module for Actor and Critic models."""

    def __init__(self, state_size, output_size, seed, fc1_units, fc2_units):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            output (int): Dimension of the output
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(PPOModel, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, output_size)


In [7]:
# %load -s Actor coball/model
class Actor(PPOModel):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed=2, fc1_units=128, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__(state_size, action_size, seed, fc1_units, fc2_units)
        self._reset_parameters()

    def _reset_parameters(self):
        self.fc1.bias.data.normal_(0.0, 3e-3)
        self.fc1.weight.data.uniform_(*self._init_limits(self.fc1))
        self.fc2.bias.data.normal_(0.0, 3e-3)
        self.fc2.weight.data.uniform_(*self._init_limits(self.fc2))
        self.fc3.bias.data.normal_(0.0, 3e-3)
        self.fc3.weight.data.uniform_(0.0, 3e-3)

    def _init_limits(self, layer):
        input_size = layer.weight.data.size()[0]
        lim = 1. / np.sqrt(input_size)

        return (-lim, lim)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> action means."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        return F.tanh(self.fc3(x))


In [8]:
# %load -s Critic coball/model
class Critic(PPOModel):
    """Critic (Value) Model."""

    def __init__(self, state_size, seed=2, fc1_units=128, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Critic, self).__init__(state_size, 1, seed, fc1_units, fc2_units)
        self._reset_parameters()

    def _reset_parameters(self):
        self.fc1.weight.data.uniform_(*self._init_limits(self.fc1))
        self.fc2.weight.data.uniform_(*self._init_limits(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def _init_limits(self, layer):
        input_size = layer.weight.data.size()[0]
        lim = 1. / np.sqrt(input_size)

        return (-lim, lim)

    def forward(self, state):
        """Build an critic (value function) network that maps states -> values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        return self.fc3(x)


The policy itself is implemented in a separate class that wraps the mean approximator and provides the distribution function for each action. The *Policy* class can be used to calculate the log probabilities of given state-action pairs as well a to sample actions in a given state.

In [9]:
# %load -s Policy coball/model
class Policy():
    """Policy that provides action probabilities and samples actions accordingly.

    The policy consists of a combination of two normal distributions, one with
    high variance to drive exploration and one with low variance to drive
    exploitation. The mean of both distributions is equal and provided by the
    underlying model.
    """

    def __init__(self, model, sigma=0.01, sigma_explore=0.5, epsilon=0.1, cap=[-1.0, 1.0], replay=False):
        """Initialize parameters.
        Params
        ======
            model fn: state -> means: A model that maps states to means of the
                  action distributions
            sigma (float): variance of the action distribution for exploitation
            sigma_explore (float): variance of the action distribution exploration
            epsilon: weight of the exploration distribution
            cap (list): limits for the action values
        """
        if sigma <= 0.0:
            raise ValueError("sigma must be positive: " + str(sigma))

        self._model = model
        self._sigma = sigma
        self._sigma_explore = sigma_explore
        self._epsilon = epsilon
        self._cap_min = cap[0] if cap is not None else None
        self._cap_max = cap[1] if cap is not None else None
        self._replay = replay

    def log_prob(self, states, actions):
        """Log-probabilities of the given actions for the given states.

        Params
        ======
        states (tensor): dimensions (agents, steps, state_size)
        actions (tensor): dimensions (agents, steps, action_size)

        Returns
        ======
        log probabilities (tensor): dimensions (agents, steps, 1)
        """
        if not states.dim() == actions.dim() == 3:
            raise ValueError("dimensions are too small, unsqueeze: " + str(states.size()) + "-" + str(actions.size()))

        if not states.size()[:2] == actions.size()[:2]:
            raise ValueError("dimenions don't match: " + str(states.size()) + "-" + str(actions.size()))

        states = states.to(device, dtype=torch.float)
        actions = actions.to(device, dtype=torch.float)

        dist_exploit, dist_explore = self.distributions(states)

        prob_exploit = torch.exp(dist_exploit.log_prob(actions).float() + math.log(1 - self._epsilon))
        prob_explore = torch.exp(dist_explore.log_prob(actions).float() + math.log(self._epsilon))
        log_probs = torch.log(prob_exploit + prob_explore)

        if self._cap_max is not None:
            boundary = torch.ge(actions, self._cap_max)
            if boundary.any():
                cdf = (1 - self._epsilon) * (1 - dist_exploit.cdf(actions).float()) \
                    + self._epsilon * (1 - dist_explore.cdf(actions).float())
                log_cdf = torch.log(cdf).float()
                log_probs = torch.where(boundary, log_cdf, log_probs)
        if self._cap_min is not None:
            boundary = torch.le(actions, self._cap_min)
            if boundary.any():
                cdf = (1 - self._epsilon) * dist_exploit.cdf(actions).float() \
                        + self._epsilon * 1 - dist_explore.cdf(actions).float()
                log_cdf = torch.log(cdf).float()
                log_probs = torch.where(boundary, log_cdf, log_probs)

        return torch.sum(log_probs, dim=2).unsqueeze(2)

    def distributions(self, states):
        means = self._model(states.to(device, dtype=torch.float))

        return (dist.Normal(means, self._sigma), dist.Normal(means, self._sigma_explore))

    def sample(self, states):
        """Sample actions for the given states according to the policy.

        Params
        ======
        states (tensor): dimensions (agents, steps, state_size)

        Returns
        ======
        actions (tensor): dimensions (agents, steps, action_size)
        """
        if self._replay:
            return self._model(states.to(device, dtype=torch.float))

        dist_exploit, dist_explore = self.distributions(states)
        if self._epsilon == 0.0 or random.uniform(0, 1) > self._epsilon:
            sample = dist_exploit.sample()
        else:
            sample = dist_explore.sample()

        if self._cap_min is None or self._cap_max is None:
            return sample
        else:
            return torch.clamp(sample, self._cap_min, self._cap_max)


### 4.4 PPO implementation

The implementation of the algorithm is contained in the *PPOLearner* class that executes the the PPO algorithm with the given paramters. It also takes care of concatenating episodes and windowing: 

In [11]:
# %load -s PPOLearner coball/training
class PPOLearner():
    """Implementation of the PPO algorithm.
    """

    def __init__(self, env=None,
            window_size=72, window_step=2, rollout_length=1152,
            episodes_in_epoch=8, ppo_epochs=2, batch_size=32,
            sigma=0.1, sigma_explore=0.5, epsilon=0.25,
            ppo_clip=0.1, gamma=1.0, gae_tau=0.1, lr=1e-3, grad_clip=1):
        # Don't instantiate as default as the constructor already starts the unity environment
        self._env = env if env is not None else CoBallEnv()

        self._state_size = self._env.get_state_size()
        self._actions = self._env.get_action_size()

        self._rollout_length = rollout_length
        self._episodes_in_epoch = episodes_in_epoch
        self._ppo_epochs = ppo_epochs
        self._batch_size = batch_size

        self._window_size = window_size
        self._window_step = window_step

        self._sigma = sigma
        self._sigma_explore = sigma_explore
        self._epsilon = epsilon
        self._gamma = gamma
        self._gae_tau = gae_tau

        self._ppo_clip = ppo_clip
        self._grad_clip = grad_clip
        self._lr = lr

        self._policy_model = Actor(self._state_size, self._actions).to(device)
        self._value_model = Critic(self._state_size).to(device)
        self._set_learning_rate(lr)

        self._policy_model.eval()
        self._value_model.eval()

        print("Initialize PPOLearner with model:")
        print(self._policy_model)
        print(self._value_model)

    def _set_learning_rate(self, lr):
        self._policy_optimizer = optim.Adam(self._policy_model.parameters(), lr, eps=1e-5)
        self._value_optimizer = optim.Adam(self._value_model.parameters(), lr, eps=1e-5)

    def _adapt(self, epoch):
        if epoch == 96:
            self._set_learning_rate(self._lr/2)
        if epoch == 128:
            self._window_size = 96
        if epoch == 256:
            self._window_size = 128

    def save(self, path, dir="results"):
        """Store the learning result.

        Store the parameters of the current models to the given path.
        """
        torch.save(self._policy_model.state_dict(), os.path.join(dir, "actor_" + path))
        torch.save(self._value_model.state_dict(), os.path.join(dir, "critic_" + path))

    def load(self, path, dir="results"):
        """Load learning results.

        Load the parameters from the given path into the models.
        """
        self._policy_model.load_state_dict(torch.load(os.path.join(dir, "actor_" + path)))
        self._value_model.load_state_dict(torch.load(os.path.join(dir, "critic_" + path)))
        self._policy_model.to(device)

    def get_agent(self, epsilon=0.0, replay=False):
        """Return an agent based on the current policy model.
        """
        return CoBallAgent(self.get_policy(epsilon=epsilon, replay=replay))

    def get_policy(self, epsilon=0.0, replay=False):
        """Return a policy based on the model of the learner.
        """
        return Policy(self._policy_model, sigma=self._sigma,
                sigma_explore=self._sigma_explore, epsilon=self._epsilon, replay=replay)

    def train(self, num_epochs=100):
        for epoch in range(num_epochs):
            self._adapt(epoch)

            policy = self.get_policy(self._epsilon)

            episodes = ( self._generate_episode(policy, epoch) for i in range(self._episodes_in_epoch) )
            episodes = ( e for e in zip(*episodes) )

            states, actions, rewards, next_states, is_terminals = \
                    [ torch.cat(component, dim=0).detach() for component in episodes ]

            returns = self._calculate_returns(rewards).detach()
            log_probs = policy.log_prob(states, actions).detach()
            values = self._value_model(states).detach()
            next_values = self._value_model(next_states).detach()

            td_errors = rewards + self._gamma * next_values - values
            advantages = self._calculate_advantages(td_errors)

            advantages = (advantages - advantages.mean()) / advantages.std()

            print("Collected windows: " + str(states.size()))

            # Validate dimensions
            for data in (states, actions, next_states, rewards, is_terminals,
                    returns, advantages, values, log_probs):
                assert len(set([ d.size()[0] for d in data ])) == 1
            for data in (states, actions, next_states, rewards, is_terminals,
                    returns, advantages, values, log_probs):
                assert len(set([ d.size()[1] for d in data ])) == 1
            assert self._env.get_state_size() == states.size()[2] == next_states.size()[2]
            assert self._env.get_action_size() == actions.size()[2]
            for data in (rewards, is_terminals, returns, advantages, values, log_probs):
                assert 1 == data.size()[2]

            self._value_model.train()
            self._policy_model.train()

            for ppo_epoch in range(self._ppo_epochs):
                sampler = [idx for idx in self._random_sample(np.arange(states.size(0)), self._batch_size) ]
                for batch_indices in sampler:
                    try:
                        with autograd.detect_anomaly():
                            batch_indices = torch.tensor(batch_indices).long()
                            sampled_states, sampled_actions, sampled_log_probs_old, sampled_returns, sampled_advantages = [
                                    data[batch_indices]
                                    for data in [states, actions, log_probs, returns, advantages] ]

                            new_log_probs = policy.log_prob(sampled_states.detach(), sampled_actions.detach())
                            ratio = (new_log_probs - sampled_log_probs_old.detach()).exp()

                            obj = ratio * sampled_advantages.detach()
                            obj_clipped = ratio.clamp(1.0 - self._ppo_clip, 1.0 + self._ppo_clip) \
                                    * sampled_advantages.detach()
                            policy_loss = - torch.min(obj, obj_clipped).mean()
                            # if policy_loss != -obj.detach().mean():
                            #     print("clipped")

                            new_value = self._value_model(sampled_states.detach())
                            value_loss = 0.5 * (sampled_returns.detach() - new_value).pow(2).mean()

                            self._value_optimizer.zero_grad()
                            value_loss.backward()
                            nn.utils.clip_grad_norm_(self._value_model.parameters(), self._grad_clip)
                            self._value_optimizer.step()

                            self._policy_optimizer.zero_grad()
                            policy_loss.backward()
                            nn.utils.clip_grad_norm_(self._value_model.parameters(), self._grad_clip)
                            self._policy_optimizer.step()
                    except BaseException as e:
                        pass

                self._value_model.eval()
                self._policy_model.eval()

                yield policy_loss, self._env.get_score(), ppo_epoch == self._ppo_epochs - 1

    def _generate_episode(self, policy, epoch):
        agent = CoBallAgent(policy)

        episode = ( step_data for step_data in self._env.generate_episode(agent,
                max_steps=self._rollout_length, episodic=False, train_mode=True) )
        episode = ( episode_data for episode_data in zip(*episode) )

        # state = tuple of (1,33) arrays, etc.., concat along first dimension
        states, actions, rewards, next_states, is_terminals = [
                torch.stack(tuple(self._to_tensor(step)[0] for step in data), dim=1)
                for data in episode ]

        assert self._env.get_agent_size() == states.size()[0]

        #split episodes
        states, actions, rewards, next_states, is_terminals = [
                self._split(data, self._window_size)
                for data in [states, actions, rewards, next_states, is_terminals] ]

        positive_rewards = torch.any(rewards > 0.0, dim=1).squeeze()
        states, actions, rewards, next_states, is_terminals = [
                data[positive_rewards,:,:]
                for data in [states, actions, rewards, next_states, is_terminals] ]

        # Verify dimensions
        assert states.size()[0] == actions.size()[0] == rewards.size()[0] \
                == next_states.size()[0] == is_terminals.size()[0]
        assert self._env.get_state_size() == states.size()[2] == next_states.size()[2]
        assert self._env.get_action_size() == actions.size()[2]
        assert 1 == rewards.size()[2] == is_terminals.size()[2]

        print("Generated episode: " + str(states.size()[0]) + "/" + str(len(positive_rewards)) \
                + " windows (" + str(states.size()[1]) + ")")

        return (states, actions, rewards, next_states, is_terminals) if states.nelement() > 0 \
                else self._generate_episode(policy, epoch)

    def _cat_component(self, episodes, component):
        return torch.cat(episodes[component], dim=0)

    def _split(self, x, split_size):
        if x.size()[1] % split_size != 0:
            raise ValueError("Illegal state, episode cannot be split: " + str(x.size()))

        splits = x.size()[1] // split_size
        step = self._window_step

        windows = x.view(splits*x.size()[0], split_size, x.size()[2])
        shifted_windows = tuple([ x[:,i:-split_size+i,:].contiguous()
                .view((splits-1)*x.size()[0], split_size, x.size()[2])
                for i in range(step, split_size, step) ])

        return torch.cat((windows,) + shifted_windows, dim=0)

    def _to_tensor(self, *arrays, dtype=torch.float):
        results = [ torch.tensor(a).to(device, dtype=dtype) if not torch.is_tensor(a) else a
                for a in arrays ]

        return tuple(result.unsqueeze(dim=1) if result.dim() == 1 else result
                for result in results)

    def _random_sample(self, indices, batch_size):
        indices = np.asarray(np.random.permutation(indices))
        batches = indices[:len(indices) // batch_size * batch_size].reshape(-1, batch_size)
        for batch in batches:
            yield batch
        r = len(indices) % batch_size
        if r:
            yield indices[-r:]

    def _calculate_returns(self, rewards):
        flipped = torch.flip(rewards, dims=(1,))
        result = torch.zeros_like(flipped)
        result[:,0,:] = flipped[:, 0, :]
        for i in range(1, flipped.size()[1]):
            result[:,i,:] = self._gamma * result[:,i-1,:] + flipped[:,i,:]

        return torch.flip(result, dims=(1,))

    def _calculate_advantages(self, td_errors):
        flipped = torch.flip(td_errors, dims=(1,))
        result = torch.zeros_like(flipped)
        result[:,0,:] = flipped[:, 0, :]
        for i in range(1, flipped.size()[1]):
            result[:,i,:] = self._gamma * self._gae_tau * result[:,i-1,:] + flipped[:,i,:]

        return torch.flip(result, dims=(1,))


### 4.5 Training
Now we can train the agent. To solve the environment we need to train the agent about 250 epochs:

In [ ]:
from coball.util import print_progress

episode_cnt = 0
episode_step = 0
for cnt, data in enumerate(PPOLearner(env=env).train(250)):
    episode_step += 1
    performance, score, terminal = data

    if terminal:
        episode_cnt += 1
        episode_step = 0

    print_progress(episode_cnt, episode_step, performance.item(), env.get_score_history(), total=2)
    if terminal:
        print("")

We leave the execution of the training in this notebook to the reader, but be aware that it might take about 2 hours or more. A logfile of a training run with 400 epochs can be found [here](results/training.log).


### 4.6 Evaluation

The related score plot during the learning process looks like this:

![Scores](scores.png)

**As shown in the plot, the agent reaches an average score of +0.3 on 100 episodes after about than 25000 episodes during training. The trained agent achieves an average score of around 2.5 on 100 episodes when executing the policy without noise:**

### 4.6 Running the trained agent

In [12]:
learner = PPOLearner(env=env)
learner.load("parameters.pt")
replay_agent = learner.get_agent(replay=True)

for i in range(100):
    episode = env.generate_episode(replay_agent, train_mode=True)
    length = 0
    for _ in episode:
        # Consume the generated steps
        length += 1
    if i % 10 == 9:
        print("The agent just played an other episode with " + str(length) + " timesteps - be patient ;)")

print("Average score on 100 episodes: " + str(np.mean(env.get_score_history())))

Initialize PPOLearner with model:
Actor(
  (fc1): Linear(in_features=24, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)
Critic(
  (fc1): Linear(in_features=24, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)


/Users/tkb/anaconda3/envs/drlnd/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
The agent just played an other episode with 1001 timesteps - be patient ;)
Average score on 100 episodes: 2.6090000388771295


The agents mostly hit
the built in limit of 1001 time steps for an episode an achives an average of __2.6__ over 100 episodes!

Finally let's watch the trained agents playing a game! We load the stored parameter files and execute the agent. Not that again for execution we set the policy to not add noise to the estimates of the actor model, but use the estimated means directly.

In [13]:
learner = PPOLearner(env=env)
learner.load("parameters.pt")
replay_agent = learner.get_agent(replay=True)

episode = env.generate_episode(replay_agent)

for count, step_data in enumerate(episode):
    # Consume the generated steps
    pass

print("Score:    " + str(env.get_score()))

Initialize PPOLearner with model:
Actor(
  (fc1): Linear(in_features=24, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)
Critic(
  (fc1): Linear(in_features=24, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)
Score:    2.600000038743019


## 5. Appendix: Varinant of the PPO Algorithm

Given that each agent only has a local observation of the environment, i.e. it does not know the state of the other agent we followed a naive approach to the problem and just applied a traditional reinforcement algorithm to the problem, which turned out to be capable of handling non-stationarity of the problem sufficiently well. The algorithm used is a variant of Proximal Policy Optimization (PPO) with a Generalized Advantage Estimation (GAE) and a clipped objective function. The algorithm uses a finite horizon instead of reward discounting. The algorithm follows closely the algorithm described in section 2 of this [project on continuous control](https://github.com/numblr/drlnd-/blob/master/Report.ipynb#2.-Algorithm). For convenience we include the description in section .... To make the algorithm work for the problem examined in this workspace the following modifications were made:

### 5.1 Proximal Policy Optimization (PPO)

Proximal Policy Optimization is a variant of policy gradient methods that is used to improve sample efficiency. Policy gradient methods are based on the objective to maximize the expectation $\mathbb{E}_{\tau \sim \pi_\theta}\left[R(\tau)\right]$ of the total reward $R(\tau)$ for trajectories $\tau=(s_0, a_0, r_0, s_1, a_1, r_1, \dots)$ under a parameterized policy $\pi_\theta(\tau)$.

The gradient of the above objective can be derived as 

$$
g
= \nabla_\theta \, \mathbb{E}_{\tau \sim \pi_\theta} \!\! \left[ \sum_{t=0}^H \log{\pi_\theta(a_t|s_t)} (R_t^f - b(s_t) \right]
= \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^H \frac{\nabla_\theta \pi_\theta(a_t|s_t)}{\pi_\theta(a_t|s_t)} (R_t^f - b(s_t) \right]
$$

where $R_f^t$ denotes the future reward at time step $t$ and $b(s_t)$ is a baseline which can depend on the state $s_t$. An equivalent formulation can be obtained by looking at the expectation with respect to a different policy $\pi_{\theta'}$:

$$
g
= \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^H \frac{\nabla_\theta \pi_\theta(a_t|s_t)}{\pi_\theta(a_t|s_t)} (R_t^f - b(s_t) \right]
= \mathbb{E}_{\tau \sim \pi_{\theta'}}  \left[ \sum_{t=0}^H \frac{\left. \nabla_\theta \pi_\theta(a_t|s_t) \right|_{\theta = \theta'}}{\pi_{\theta'}(a_t|s_t)} (R_t^f - b(s_t) \right].
$$

When the above expectations are approximated by data from sampled trajectories, this implies that we can estimate the gradient for an evolved policy $\pi_\theta$ from trajectories that were sampled using the old policy $\pi_{\theta'}$ reasonably well if $\theta$ is in a proximity of $\theta'$. In other words, we can reuse trajectories generated with $\pi_{\theta'}$ to further optimize $\pi_{\theta}$, as long as $\pi_{\theta}$ does not get too far from $\pi_{\theta'}$.
The above gradient can be obtained by diffentiating the objective function 

$$
L(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta'}}  \left[ \sum_{t=0}^H \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta'}(a_t|s_t)} (R_t^f - b(s_t) \right].
$$

As proposed in this [article](https://arxiv.org/abs/1707.06347), an easy and efficient way protect against the inacurracy introduced by a too large deviation of $\theta$ from $\theta'$ is to clip the ratio $r(\theta) = \frac{\pi_{\theta}}{\pi_{\theta'}}$ to a maximum value of $1 + \epsilon$

$$
L^{PPO}(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta'}} \left [
    \min(r(\theta)(R_t^f - b(s_t)), \rm{clip}(r(\theta), 1-\epsilon, 1+\epsilon) (R_t^f - b(s_t)))
\right],
$$

where the clipping parameter $\epsilon$ is a hyper parameter of the algorithm.


### 5.2 Generalized Advantage Estimation (GAE) Critic

As a baseline $b(s_t)$ we use the state-value function $V_{\pi_\theta}(s_t)$, and replace the term $(R_t^f - V_{\pi_\theta}(s_t))$ with a function approximator $\tilde A(s, a)$ (critic). $\tilde A(s, a)$ is an approximation of the advantage function $A_{\pi_\theta}(s, a) = Q_{\pi_\theta}(s,a) - V_{\pi_\theta}(s)$. To obtain this estimate we use the generalized advantage estimator as described in [this article](https://arxiv.org/abs/1506.02438), which approximates the advantage function using the state-value function as a weighted average over n-step temporal differences $r_t + r_{t+1} + \cdots + r_{t+n-1} + V(s_{t+n}) - V(s_{t})$:

$$
\tilde A^{GAE} = - \tilde V_{\pi_\theta}(s_t) + (1-\lambda)\sum_{k=t}^H \lambda^{(k-t)} \left(\sum_{l=t}^{k-1} r(s_l) + \tilde V(s_k)\right) = \sum_{k=t}^H \lambda^{(k-t)}\left(r_{k} + \tilde V(s_{k+1}) - \tilde V(s_{k})\right) = \sum_{k=t}^H \lambda^{(k-t)}\delta_k^{\tilde V},
$$
where $\delta_t^V = r_t + V(s_{t+1}) - V(s_{t})$.

The estimate $\tilde V$ of the state-value is obtained by minimizing the error between the estimated and sampled state-values for a paramterized estimator $\tilde V_\phi(s_{t})$:
$$
L^V = \hat{\mathbb{E}} \left[ \| R_t^f - \tilde V_\phi(s_{t}) \| \right]
$$

### 5.3 Finite horizon and noise reduction

Instead of discounting of rewards we decided to consider a fixed size finite horizon for the agent. This is implemented by generating full episodes, splitting them with a rolling window corresponding to the horizon of the agent and removing windows in which the agent does not obtain any reward. 

![windows](resources/windows.png)

The reason for this is the following: Though the task is setup to be episodic, it is continuous in it's nature, and actions should have a comparable short term effect. From this observation it seems reasonable to assume that a finite horizon should be sufficient for the agent to solve the task. Further, looking at the actor at the beginning of the learning process, it is evident that it suffers from find a good starting point, as the probability to reach the target by accident is low. Implemening a fixed horizon instead of discounting gives us an easy way to remove windows in which the agent doesn't retrieve any reward. Considering that we can understand the learning process as a directed search driven by the obtained reward, it seems reasonable to assume that these windows do not contribute in a positive way to the learning process.

### 5.3 Approximators and training

For both the approximator $\pi_\theta$ as well as the state-value function $V_\phi$ we use fully connected neural networds with two hidden layers of size 128 and 64 with relu activations. The two networks differ only in the output layers, as described in the following. The approximators are trained in batches and a replay buffer is used, i.e. the  order of timestpes is randomized during training.

#### 5.3.1 Policy approximator

The policy approximator maps a state to a normal distribution  with a fixed variance and an estimated mean for each of the available actions.  The normal distributions are truncated to the interval $[-1,1]$ to conform to the allowed values for the actions. The mean estimate is provided by a neural network as described above, where the size of the output layer corresponds to the number of available actions and uses a $\tanh$ activation function to confine the output to the interval $[-1,1]$.

![Actor net](resources/actor_net.png)

#### 5.3.2 State-Value approximator

The approximator for the state value function maps a state to a real number and is implemented by a neural network as described above with a single node in the output layer without additional activation.

![Critic net](resources/critic_net.png)

### 5.4 Algorithm:

* Init $\theta'$, $\theta$, $\phi$
* For $N$ epochs:
    * Set $\theta' = \theta$
    * Generate $M$ episodes from each agent with respect to $\pi_{\theta'}$
    * Split the episodes into windows of length $m$ using a rolling window
    * Remove windows where no reward is achived
    * Calculate $\pi_{\theta'}(a_t|s_t)$ and $A^{GAE}(s_t, a_t)$ for all time steps in each window
    * For $k$ epochs:
        * Update $\theta$ to optimize $L^{PPO}$ using sampled a batches of $j$ windows
        * Update $\phi$ to optimize $L^V$ using the same batches as in the previous step
 
